# Compute RHI Scan String

```
Input: radar (lon, lat) and heading; target (lon, lat) and height

Output elevation, azimuth, range
```

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def deg2rad(x):
    return x * np.pi / 180.

def rad2deg(x):
    return x * 180. / np.pi

def coord2point(coord, h=0.):
    (lon, lat) = coord
    r = 6371. + h
    lon_rad = deg2rad(lon)
    lat_rad = deg2rad(lat)
    return np.array([
        r * np.cos(lat_rad) * np.sin(lon_rad),
        r * np.sin(lat_rad),
        r * np.cos(lat_rad) * np.cos(lon_rad),
    ], dtype=float)

def rotx(x):
    sx = np.sin(x)
    cx = np.cos(x)
    return np.array([[1., 0., 0.], [0., cx, -sx], [0., sx, cx]])


def roty(x):
    sx = np.sin(x)
    cx = np.cos(x)
    return np.array([[cx, 0., sx], [0., 1., 0.], [-sx, 0., cx]])

def rotz(x):
    sx = np.sin(x)
    cx = np.cos(x)
    return np.array([[cx, -sx, 0.], [sx, cx, 0.], [0., 0., 1.]])

def norm(x):
    return np.sqrt(np.sum(x ** 2))

def distance(x1, x2):
    p1 = coord2point(x1)
    p2 = coord2point(x2, h=100)
    d = norm(p1 - p2)
    return d

def showvec3(p):
    (x, y, z) = p
    return f'{x:.2f} {y:.2f} {z:.2f}'

In [43]:
def params(target, radar, height=2.5, debug=False):
    pt = coord2point(target, h=height)

    ry = roty(deg2rad(-radar[0]))
    rx = rotx(deg2rad(-radar[1]))
    m = np.matmul(rx, ry)

    yt = np.matmul(m, pt)
    
    d = yt[:2]
    r = norm(d)
    az = rad2deg(np.arctan2(d[0], d[1]))
    el = rad2deg(np.arctan2(height, r))

    if debug:
        pr = coord2point(radar)
        yr = np.matmul(m, pr)

        # zenith = pr / norm(pr)
        # east = np.cross([0, 1, 0], zenith)
        # east = east / norm(east)
        # north = np.cross(zenith, east)
        # north = north / norm(north)

        print('target @ ' + showvec3(yt))
        print('radar @ ' + showvec3(yr))
        print(f'el = {el:.2f}   az = {az:.2f}   r = {r:.2f}')
        
    return {
        'el': el,
        'az': az,
        'r': r
    }

def vcp(params, heading=0.):
    az = np.round(params['az'] - heading)
    el = max(20, np.ceil(params['el'] / 2.) * 2.)
    p = f'0,{el:.0f}'
    s = 'p vol '
    for a in range(-12, 14, 2):
        ra = az + a;
        s += f'r {ra:.0f} {p}/'
    return s

In [44]:
radar = (-90, 0)
# x2 = (-91, 36)
# x2 = (-90, -1)
target = (-90.1, 0.1)

p = params(target, radar)
# v = vcp(p, heading=10.)
v = vcp(p)

print(p)
print(v)

{'el': 9.029761111966844, 'az': -44.99995636674476, 'r': 15.731488036496089}
p vol r -57 0,20/r -55 0,20/r -53 0,20/r -51 0,20/r -49 0,20/r -47 0,20/r -45 0,20/r -43 0,20/r -41 0,20/r -39 0,20/r -37 0,20/r -35 0,20/r -33 0,20/


# Lat/Lon on Earth

In [ ]:
points = []
for lat in range(0, 80, 10):
    for lon in range(-180, 180, 10):
        x = coord2point((lon, lat))
        points.append(x)
pp = np.array(points)

In [ ]:
fig = plt.figure(figsize=(5, 5), dpi=144)
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pp[:, 0], pp[:, 1], pp[:, 2], marker='.')
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.grid(False)

In [ ]:
plt.close()